In [1]:
from mxnet import nd
from mxnet.gluon import nn

In [17]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)

    def forward(self, x):
        return self.output(self.hidden(x))

In [18]:
X = nd.random.uniform(shape=(2, 20))
net = MLP()
net.initialize()
net(X)


[[-0.00631883 -0.03835293  0.00229847  0.04939309  0.02460632 -0.01067401
  -0.0559635  -0.00838179  0.02698104  0.03468554]
 [-0.03824129 -0.04491622  0.04493505  0.06538454  0.02332134 -0.0377148
  -0.05333055 -0.04294174  0.02069081  0.03468261]]
<NDArray 2x10 @cpu(0)>

In [5]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)

    def add(self, block):
        self._children[block.name] = block

    def forward(self, x):
        for block in self._children.values():
            x = block(x)
        
        return x

In [8]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(X)


[[-0.07673368 -0.027709    0.03064326 -0.03149016  0.02848468  0.10131303
   0.00625361 -0.04837743 -0.10565996 -0.07495882]
 [ 0.0001531   0.00130881 -0.0376035  -0.05839263 -0.00849807  0.06806571
  -0.01718362 -0.07539228 -0.0472235  -0.06972831]]
<NDArray 2x10 @cpu(0)>

In [10]:
X


[[0.7142413  0.46924916 0.998847   0.84114015 0.1494483  0.90464777
  0.86812603 0.03755938 0.16249293 0.50831544 0.6155596  0.16684751
  0.12381998 0.779051   0.8480082  0.8649333  0.807319   0.4113967
  0.56910074 0.13997258]
 [0.4071833  0.03322238 0.06916699 0.98257494 0.69742876 0.37329075
  0.45354268 0.42007536 0.7220556  0.05058811 0.8663823  0.3654961
  0.9755215  0.01662797 0.8558034  0.23074234 0.01171408 0.7649117
  0.35997805 0.9441235 ]]
<NDArray 2x20 @cpu(0)>

In [25]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        self.rand_weight = self.params.get_constant(
            'rand_weight', nd.random.uniform(shape=(20, 20))
        )
        self.dense = nn.Dense(20, activation='relu')

    def forward(self, x):
        x = self.dense(x)
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        x = self.dense(x)

        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10

        return x.sum()

In [26]:
net = FancyMLP()
net.initialize()
net(X)


[2.8089633]
<NDArray 1 @cpu(0)>

In [13]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                     nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')

    def forward(self, x):
        return self.dense(self.net(x))

In [14]:
net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())
net.initialize()
net(X)


[24.81585]
<NDArray 1 @cpu(0)>